In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

workout_df = pd.read_csv('workouts.csv')
workout_df

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe
0,Leg day - quarta-feira,"16 Apr 2025, 15:56","16 Apr 2025, 17:54",NaN,Squat (Barbell),NaN,NaN,0,normal,140.0,6,NaN,NaN,NaN
1,Leg day - quarta-feira,"16 Apr 2025, 15:56","16 Apr 2025, 17:54",NaN,Squat (Barbell),NaN,NaN,1,failure,145.0,4,NaN,NaN,NaN
2,Leg day - quarta-feira,"16 Apr 2025, 15:56","16 Apr 2025, 17:54",NaN,Leg Press (Machine),NaN,NaN,0,normal,165.0,8,NaN,NaN,NaN
3,Leg day - quarta-feira,"16 Apr 2025, 15:56","16 Apr 2025, 17:54",NaN,Leg Press (Machine),NaN,NaN,1,failure,170.0,7,NaN,NaN,NaN
4,Leg day - quarta-feira,"16 Apr 2025, 15:56","16 Apr 2025, 17:54",NaN,Lying Leg Curl (Machine),NaN,NaN,0,normal,45.0,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Calf Press (Machine),NaN,NaN,1,normal,180.0,10,NaN,NaN,NaN
6455,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Calf Press (Machine),NaN,NaN,2,failure,200.0,10,NaN,NaN,NaN
6456,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Cable Crunch,NaN,NaN,0,normal,70.0,14,NaN,NaN,NaN
6457,Back day - segunda-feira,"11 Dec 2023, 16:54","11 Dec 2023, 19:08",NaN,Cable Crunch,NaN,NaN,1,normal,70.0,11,NaN,NaN,NaN


In [2]:
workout_df.columns

Index(['title', 'start_time', 'end_time', 'description', 'exercise_title',
       'superset_id', 'exercise_notes', 'set_index', 'set_type', 'weight_kg',
       'reps', 'distance_km', 'duration_seconds', 'rpe'],
      dtype='object')

In [3]:
workout_df.dtypes

title                object
start_time           object
end_time             object
description          object
exercise_title       object
superset_id         float64
exercise_notes       object
set_index             int64
set_type             object
weight_kg           float64
reps                  int64
distance_km         float64
duration_seconds    float64
rpe                 float64
dtype: object

In [4]:
workout_df.describe()

,superset_id,set_index,weight_kg,reps,distance_km,duration_seconds,rpe
count,48.0,6459.000000,6447.000000,6459.000000,0.0,0.0,0.0
mean,0.0,1.163493,70.024556,6.821180,NaN,NaN,NaN
std,0.0,1.099097,54.561507,2.526186,NaN,NaN,NaN
min,0.0,0.000000,1.000000,1.000000,NaN,NaN,NaN
25%,0.0,0.000000,30.000000,5.000000,NaN,NaN,NaN
50%,0.0,1.000000,50.000000,7.000000,NaN,NaN,NaN
75%,0.0,2.000000,100.000000,8.000000,NaN,NaN,NaN
max,0.0,6.000000,330.000000,69.000000,NaN,NaN,NaN


In [5]:
workout_df_columns_filtered = workout_df[['title','start_time', 'exercise_title', 'set_index', 'set_type', 'weight_kg', 'reps']]
workout_df_columns_filtered = workout_df_columns_filtered.rename(columns={'start_time': 'date'})
workout_df_columns_filtered


,title,date,exercise_title,set_index,set_type,weight_kg,reps
0,Leg day - quarta-feira,"16 Apr 2025, 15:56",Squat (Barbell),0,normal,140.0,6
1,Leg day - quarta-feira,"16 Apr 2025, 15:56",Squat (Barbell),1,failure,145.0,4
2,Leg day - quarta-feira,"16 Apr 2025, 15:56",Leg Press (Machine),0,normal,165.0,8
3,Leg day - quarta-feira,"16 Apr 2025, 15:56",Leg Press (Machine),1,failure,170.0,7
4,Leg day - quarta-feira,"16 Apr 2025, 15:56",Lying Leg Curl (Machine),0,normal,45.0,9
...,...,...,...,...,...,...,...
6454,Back day - segunda-feira,"11 Dec 2023, 16:54",Calf Press (Machine),1,normal,180.0,10
6455,Back day - segunda-feira,"11 Dec 2023, 16:54",Calf Press (Machine),2,failure,200.0,10
6456,Back day - segunda-feira,"11 Dec 2023, 16:54",Cable Crunch,0,normal,70.0,14
6457,Back day - segunda-feira,"11 Dec 2023, 16:54",Cable Crunch,1,normal,70.0,11


In [6]:
workout_df_columns_filtered['date'] = pd.to_datetime(workout_df_columns_filtered['date'], format="%d %b %Y, %H:%M")
# workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date
workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date
# Data atual
hoje = datetime.now().date()

# Data limite
limite = hoje - timedelta(days=90)

# Filtrando
workout_df_columns_filtered = workout_df_columns_filtered[workout_df_columns_filtered['date'] >= limite]
workout_df_columns_filtered.head()

,title,date,exercise_title,set_index,set_type,weight_kg,reps
0,Leg day - quarta-feira,2025-04-16,Squat (Barbell),0,normal,140.0,6
1,Leg day - quarta-feira,2025-04-16,Squat (Barbell),1,failure,145.0,4
2,Leg day - quarta-feira,2025-04-16,Leg Press (Machine),0,normal,165.0,8
3,Leg day - quarta-feira,2025-04-16,Leg Press (Machine),1,failure,170.0,7
4,Leg day - quarta-feira,2025-04-16,Lying Leg Curl (Machine),0,normal,45.0,9


In [7]:
workout_df_columns_filtered[(workout_df_columns_filtered['title'] == 'Legs sábado') & (workout_df_columns_filtered['exercise_title'] == 'Deadlift (Barbell)')]

,title,date,exercise_title,set_index,set_type,weight_kg,reps
42,Legs sábado,2025-04-12,Deadlift (Barbell),0,normal,180.0,6
43,Legs sábado,2025-04-12,Deadlift (Barbell),1,failure,185.0,5
113,Legs sábado,2025-04-05,Deadlift (Barbell),0,normal,180.0,5
114,Legs sábado,2025-04-05,Deadlift (Barbell),1,failure,180.0,6
184,Legs sábado,2025-03-29,Deadlift (Barbell),0,normal,200.0,3
185,Legs sábado,2025-03-29,Deadlift (Barbell),1,failure,180.0,5
255,Legs sábado,2025-03-22,Deadlift (Barbell),0,normal,200.0,4
256,Legs sábado,2025-03-22,Deadlift (Barbell),1,failure,200.0,3
330,Legs sábado,2025-03-15,Deadlift (Barbell),0,normal,200.0,5
331,Legs sábado,2025-03-15,Deadlift (Barbell),1,failure,200.0,3


In [8]:
import plotly.graph_objects as go
import pandas as pd

def gerar_grafico_interativo(df, treino_nome, exercicio_nome, output_path='grafico_treino.html'):
    # Cria uma cópia do DataFrame para evitar modificações no original
    df = df.copy()
    
    # Garante que a coluna 'date' está no formato datetime.date
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Filtra pelo exercício e treino desejado
    exercicio_df = df[(df['title'] == treino_nome) & (df['exercise_title'] == exercicio_nome)]

    # Verifica se há dados após o filtro
    if exercicio_df.empty:
        print(f"Nenhum dado encontrado para {exercicio_nome} em {treino_nome}")
        return None

    # Calcula a data máxima para referência
    max_date = exercicio_df['date'].max()

    # Filtra inicialmente para os últimos 30 dias (padrão)
    initial_data = exercicio_df[exercicio_df['date'] >= (max_date - pd.Timedelta(days=30))]

    # Cria os traces iniciais para os últimos 30 dias
    traces = []
    for set_idx in sorted(initial_data['set_index'].unique()):  # Ordena para consistência
        set_data = initial_data[initial_data['set_index'] == set_idx]
        traces.append(go.Scatter(
            x=set_data['date'],
            y=set_data['weight_kg'],
            mode='lines+markers+text',
            name=f'Série {set_idx + 1}',
            text=[f'{r} reps' for r in set_data['reps']],
            textposition='top center',
            hovertemplate='Data: %{x}<br>Peso: %{y} kg<br>Reps: %{text}<extra></extra>'
        ))

    # Layout com botões de filtro
    layout = go.Layout(
        title=f'Evolução no {exercicio_nome} ({treino_nome})',
        xaxis=dict(title='Data', tickformat='%d/%m/%Y'),
        yaxis=dict(title='Peso (kg)'),
        hovermode='closest',
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": "Últimos 30 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=30)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Últimos 60 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=60)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Últimos 90 dias",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[(exercicio_df['set_index'] == idx) & (exercicio_df['date'] >= (max_date - pd.Timedelta(days=90)))]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    },
                    {
                        "label": "Todos os dados",
                        "method": "restyle",
                        "args": [{
                            "x": [exercicio_df[exercicio_df['set_index'] == idx]['date'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "y": [exercicio_df[exercicio_df['set_index'] == idx]['weight_kg'] for idx in sorted(exercicio_df['set_index'].unique())],
                            "text": [exercicio_df[exercicio_df['set_index'] == idx]['reps'].apply(lambda r: f'{r} reps') for idx in sorted(exercicio_df['set_index'].unique())],
                            "visible": [True] * len(exercicio_df['set_index'].unique())
                        }],
                        "args2": [{"visible": [True] * len(exercicio_df['set_index'].unique())}],
                    }
                ],
                "direction": "down",
                "showactive": True,
                "active": 0,  # Define "Últimos 30 dias" como o botão ativo por padrão
                "x": 0.1,
                "xanchor": "left",
                "y": 1.2,
                "yanchor": "top"
            }
        ]
    )

    # Cria a figura
    fig = go.Figure(data=traces, layout=layout)
    
    # Salva o gráfico como HTML
    fig.write_html(output_path, auto_open=False)  # auto_open=False para evitar abrir o navegador automaticamente
    
    # Retorna a figura para visualização direta em notebooks
    return fig

# Exemplo de uso:
graf = gerar_grafico_interativo(workout_df_columns_filtered, 'Legs sábado', 'Deadlift (Barbell)')
if graf is not None:
    graf.show()  # Exibe o gráfico diretamente no Jupyter Notebook

In [9]:
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta

# Função auxiliar para criar os botões de séries
def create_series_buttons(orders_periodo, all_orders, has_failure):
    buttons = []
    # Top Set
    if has_failure:
        buttons.append({
            "label": "Top Set (Failure)",
            "method": "update",
            "args": [{"visible": [True] + [False]*len(all_orders)}]
        })

    # Work Sets
    for o in orders_periodo:
        buttons.append({
            "label": f"Work Set {int(o)}",
            "method": "update",
            "args": [{"visible": [False] + [True if o == current else False for current in all_orders]}]
        })

    # Todas
    buttons.append({
        "label": "Todas as séries",
        "method": "update",
        "args": [{"visible": [True] + [True if o in orders_periodo else False for o in all_orders]}]
    })

    return buttons

# Função principal com lógica dos botões
def gerar_grafico_interativo_com_series(df, treino_nome, exercicio_nome, output_path='grafico_treino_com_series.html'):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date']).dt.date

    exercicio_df = df[(df['title'] == treino_nome) & (df['exercise_title'] == exercicio_nome)]
    if exercicio_df.empty:
        print("Sem dados.")
        return None

    has_failure = not exercicio_df[exercicio_df['set_type'] == 'failure'].empty
    max_date = exercicio_df['date'].max()

    exercicio_df['normal_order'] = None
    for dt in exercicio_df['date'].unique():
        subset = exercicio_df[(exercicio_df['date'] == dt) & (exercicio_df['set_type'] == 'normal')].sort_values('set_index')
        for i, idx in enumerate(subset.index, 1):
            exercicio_df.loc[idx, 'normal_order'] = i

    exercicio_df['normal_order'] = exercicio_df['normal_order'].astype('Int64')

    all_orders = sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= (max_date - timedelta(days=90)))]['normal_order'].unique())

    # Criação de traces
    traces = []

    # Top set (visível inicialmente)
    if has_failure:
        fdata = exercicio_df[(exercicio_df['set_type'] == 'failure') & (exercicio_df['date'] >= max_date - timedelta(days=30))]
        traces.append(go.Scatter(
            x=fdata['date'], y=fdata['weight_kg'], text=[f"{r} reps" for r in fdata['reps']],
            mode='lines+markers+text', textposition='top center',
            name='Top Set (Failure)', visible=True
        ))

    # Work sets (invisíveis inicialmente)
    for o in all_orders:
        wdata = exercicio_df[(exercicio_df['normal_order'] == o) & (exercicio_df['date'] >= max_date - timedelta(days=30))]
        traces.append(go.Scatter(
            x=wdata['date'], y=wdata['weight_kg'], text=[f"{r} reps" for r in wdata['reps']],
            mode='lines+markers+text', textposition='top center',
            name=f'Work Set {int(o)}', visible=False
        ))

    # Armazena botões e dados por período
    periodos = [30, 60, 90]
    period_buttons = []

    for dias in periodos:
        periodo_inicio = max_date - timedelta(days=dias)
        orders = sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= periodo_inicio)]['normal_order'].unique())

        x_list, y_list, text_list = [], [], []

        if has_failure:
            df_f = exercicio_df[(exercicio_df['set_type'] == 'failure') & (exercicio_df['date'] >= periodo_inicio)]
            x_list.append(df_f['date'].tolist())
            y_list.append(df_f['weight_kg'].tolist())
            text_list.append([f"{r} reps" for r in df_f['reps']])
        else:
            x_list.append([]); y_list.append([]); text_list.append([])

        for o in all_orders:
            df_w = exercicio_df[(exercicio_df['normal_order'] == o) & (exercicio_df['date'] >= periodo_inicio)]
            x_list.append(df_w['date'].tolist())
            y_list.append(df_w['weight_kg'].tolist())
            text_list.append([f"{r} reps" for r in df_w['reps']])

        period_buttons.append(dict(
            label=f"Últimos {dias} dias",
            method="update",
            args=[
                {"x": x_list, "y": y_list, "text": text_list},
                {"updatemenus[1].buttons": create_series_buttons(orders, all_orders, has_failure)}
            ]
        ))
    period_buttons[0]["args"][1]["title"] = f"Evolução no {exercicio_nome} ({treino_nome}) - Últimos 30 dias"

    layout = go.Layout(
    title=f'Evolução no {exercicio_nome} ({treino_nome})',
    xaxis=dict(
        title='Data',
        tickformat='%d/%m/%Y'
    ),
    yaxis=dict(
        title='Peso (kg)'
    ),
    hovermode='closest',
    updatemenus=[
        dict(
            buttons=period_buttons,
            direction="down",
            showactive=True,
            active=0,
            x=0.6,  # Mais próximo da borda esquerda
            xanchor="left",
            y=1.2,   # Bem acima do gráfico
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1
        ),
        dict(
            buttons=create_series_buttons(
                sorted(exercicio_df[(exercicio_df['normal_order'].notna()) & (exercicio_df['date'] >= max_date - timedelta(days=30))]['normal_order'].unique()),
                all_orders,
                has_failure
            ),
            direction="down",
            showactive=True,
            active=0,
            x=0.99,  # Lado direito
            xanchor="right",
            y=1.2,   # Mesmo nível do outro menu
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1
        )
    ]
)

    fig = go.Figure(data=traces, layout=layout)
    #fig.write_html(output_path, auto_open=False)
    #return fig

    # Salva o gráfico como HTML interativo em arquivo (como já faz)
    fig.write_html(output_path, auto_open=False)

    # Também retorna o HTML como string, para inserção em JS ou <div>
    html_string = fig.to_html(full_html=False, include_plotlyjs='cdn')
    return fig, html_string

# Exemplo de uso
graf, _ = gerar_grafico_interativo_com_series(workout_df_columns_filtered, 'Legs sábado', 'Deadlift (Barbell)')
if graf is not None:
    graf.show()


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# Filtrar os exercícios únicos para o treino "Legs sábado"
exercicios_legs_sabado = workout_df_columns_filtered[workout_df_columns_filtered['title'] == 'Legs sábado']['exercise_title'].unique()
def gerar_grafico_treino_total(treino_exercicios, nome_treino='Legs sábado'):
    # Filtra os dados para o treino específico
    print(f"Exercícios na rotina {treino_exercicios}:")
    for exercicio in treino_exercicios:
        #print(f"- {exercicio}")
        graf = gerar_grafico_interativo_com_series(workout_df_columns_filtered, nome_treino, exercicio, output_path='grafico_treino.html')
        #if graf is not None:
            #graf.show()
gerar_grafico_treino_total(exercicios_legs_sabado)

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Exercícios na rotina ['Deadlift (Barbell)' 'Extensão Quadril Banco Romano'
 'Seated Leg Curl (Machine)' 'Leg Extension (Machine)' 'Seated Calf Raise']:


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [11]:
treino_nomes = workout_df_columns_filtered['title'].unique()
for nome_treino in treino_nomes:
    #print(f"\nTreino: {nome_treino}")
    # Filtrar os exercícios únicos para o treino atual
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == nome_treino]['exercise_title'].unique()
    gerar_grafico_treino_total(exercicios, nome_treino=nome_treino)
# Salvar o gráfico como HTML

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Exercícios na rotina ['Squat (Barbell)' 'Leg Press (Machine)' 'Lying Leg Curl (Machine)'
 'Leg Extension (Machine)' 'Calf Extension (Machine)' 'Cable Crunch']:


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Exercícios na rotina ['Bench Press (Barbell)' 'Seated Chest Flys (Cable)'
 'Seated Shoulder Press (Machine)' 'Single Arm Lateral Raise (Cable)'
 'Skullcrusher (Barbell)' 'Upright Row (Barbell)']:


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Exercícios na rotina ['Seated Cable Row - Bar Wide Grip' 'Pull Up (Weighted)'
 'pull around Ilíaco' 'Chest Supported Y Raise (Dumbbell)'
 'Single Arm Curl (Cable)' 'Hammer Curl (Dumbbell)' 'Cable Crunch']:


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Exercícios na rotina ['Deadlift (Barbell)' 'Extensão Quadril Banco Romano'
 'Seated Leg Curl (Machine)' 'Leg Extension (Machine)' 'Seated Calf Raise']:
Exercícios na rotina ['Single Arm Lat Pulldown' 'Chest Fly (Machine)'
 'Single Arm Lateral Raise (Cable)' 'Incline Chest Press (Machine)'
 'Seated Cable Row - V Grip (Cable)' 'Preacher Curl (Machine)'
 'Triceps Rope Pushdown' 'Cable Crunch']:


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [12]:
graficos_por_exercicio = {}
for rotina in workout_df_columns_filtered['title'].unique():
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == rotina]['exercise_title'].unique()
    for exercicio in exercicios:
        fig, html = gerar_grafico_interativo_com_series(workout_df_columns_filtered, rotina, exercicio)
        if html:
            graficos_por_exercicio[(rotina, exercicio)] = html.replace("`", "\\`")  # escapando crase

graficos_por_exercicio


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

{('Leg day - quarta-feira',
  'Squat (Barbell)'): '<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-3.0.1.min.js"></script>                <div id="f0b696de-a793-4045-a0e4-7a02e2b4b12a" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                window.PLOTLYENV=window.PLOTLYENV || {};                                if (document.getElementById("f0b696de-a793-4045-a0e4-7a02e2b4b12a")) {                    Plotly.newPlot(                        "f0b696de-a793-4045-a0e4-7a02e2b4b12a",                        [{"mode":"lines+markers+text","name":"Top Set (Failure)","text":["4 reps","5 reps","2 reps","6 reps","6 reps"],"textposition":"top center","visible":true,"x":["2025-04-16","2025-04-09","2025-04-02","2025-03-26","2025-03-19"],"y":{"dtype":"f8","bdata":"AAAAAAAgYkAAAAAAAIBhQAAAAAAAAGRAAAAAAABg

In [13]:
import json

def gerar_graficos_dict(df, rotinas_exercicios):
    graficos_dict = {}

    for rotina, exercicios in rotinas_exercicios.items():
        for exercicio in exercicios:
            fig, _ = gerar_grafico_interativo_com_series(df, rotina, exercicio)
            if fig:
                key = f"{rotina}||{exercicio}"
                graficos_dict[key] = fig.to_dict()
    
    return graficos_dict


In [14]:
rotinas_exercicios = {}
for nome_treino in treino_nomes:
    exercicios = workout_df_columns_filtered[workout_df_columns_filtered['title'] == nome_treino]['exercise_title'].unique().tolist()
    rotinas_exercicios[nome_treino] = exercicios

graficos_plotly_dict = gerar_graficos_dict(workout_df_columns_filtered, rotinas_exercicios)
from plotly.utils import PlotlyJSONEncoder

js_plotly_data = f"const graficosPlotly = {json.dumps(graficos_plotly_dict, cls=PlotlyJSONEncoder)};"


C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikol\AppData\Local\Temp\ipykernel_7496\709765694.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [23]:
html_template = f"""
<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Visualização de Treinos</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f4f6f8;
            color: #333;
            margin: 0;
            padding: 0;
        }}

        header {{
            background-color: #2d89ef;
            color: white;
            padding: 20px;
            text-align: center;
        }}

        .container {{
            max-width: 900px;
            margin: 30px auto;
            padding: 20px;
            background-color: white;
            box-shadow: 0 0 10px rgba(0,0,0,0.05);
            border-radius: 8px;
        }}

        select {{
            margin: 10px 0;
            padding: 8px;
            font-size: 16px;
            width: 100%;
            border: 1px solid #ccc;
            border-radius: 4px;
        }}

        #grafico-container {{
            width: 100%;
            height: 600px;
            margin-top: 20px;
        }}

        .alert {{
            background-color: #fff4e5;
            color: #8a6d3b;
            border: 1px solid #f0c36d;
            padding: 15px;
            border-radius: 5px;
            margin-bottom: 20px;
            font-size: 15px;
        }}

        label {{
            font-weight: bold;
            display: block;
            margin-top: 10px;
        }}
    </style>
</head>
<body>
    <header>
        <h1>Visualização de Progresso nos Treinos</h1>
    </header>

    <div class="container">
        <div class="alert">
            <strong>Importante:</strong> Após selecionar a rotina de treino e o exercício desejado, o gráfico será carregado com dados genéricos e incorretos. 
            Para garantir a exibição correta dos dados, selecione um período (ex: 30, 60 ou 90 dias) usando o botão de filtros de período no canto superior direito do gráfico.
            <br></br>Além do mais, você pode alternar entre as séries de treino (Top Set e Work Sets) usando o botão de filtros de séries no canto superior direito do gráfico, ao lado do botão de seleção de período.
            <br></br>Recomenda-se acessar esse documento em um computador para melhor visualização, caso não seja possível e você esteja utilizando um celular, habilite a opção de versão desktop no seu navegador.
        </div>

        <label for="rotina">Rotina de Treino:</label>
        <select id="rotina" onchange="atualizarExercicios()">
            <option value="">Selecione uma rotina</option>
        </select>

        <label for="exercicio">Exercício:</label>
        <select id="exercicio" onchange="carregarGrafico()">
            <option value="">Selecione um exercício</option>
        </select>

        <div id="grafico-container"></div>
    </div>

    <script>
        // Dados JS dos gráficos
        {js_plotly_data}

        // Dicionário de rotinas e exercícios
        const rotinasExercicios = {json.dumps(rotinas_exercicios)};

        const rotinaSelect = document.getElementById('rotina');
        for (const rotina in rotinasExercicios) {{
            const option = document.createElement('option');
            option.value = rotina;
            option.text = rotina;
            rotinaSelect.appendChild(option);
        }}

        function atualizarExercicios() {{
            const rotina = rotinaSelect.value;
            const exercicioSelect = document.getElementById('exercicio');
            exercicioSelect.innerHTML = '<option value="">Selecione um exercício</option>';
            if (rotinasExercicios[rotina]) {{
                rotinasExercicios[rotina].forEach(exercicio => {{
                    const option = document.createElement('option');
                    option.value = exercicio;
                    option.text = exercicio;
                    exercicioSelect.appendChild(option);
                }});
            }}
            carregarGrafico();
        }}

        function carregarGrafico() {{
            const rotina = rotinaSelect.value;
            const exercicio = document.getElementById('exercicio').value;
            const chave = rotina + "||" + exercicio;
            const container = document.getElementById('grafico-container');
            container.innerHTML = ""; 
            if (graficosPlotly[chave]) {{
                const fig = graficosPlotly[chave];
                Plotly.newPlot(container, fig.data, fig.layout);
            }}
        }}

        atualizarExercicios();
    </script>
</body>
</html>
"""


with open("visualizacao_treino_interativa.html", "w", encoding="utf-8") as f:
    f.write(html_template)
